In [65]:
from torchPIV.PIVbackend import ToTensor, PIVDataset, moving_window_array
from SIV_library.lib import block_match, moving_reference_array, correlation_to_displacement, get_field_shape, get_x_y

import os
import sys
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

dataset = PIVDataset("../Test Data/plume simulation_PROCESSED", ".jpg",
                     "sequential", transform=ToTensor(dtype=torch.uint8))

In [59]:
def vector_field_l(foldername: str, results: np.ndarray, scale: float, playback_fps = 60.) -> None:
    fig, ax = plt.subplots()

    velocities = results[:, 2:]
    abs_velocities = np.sqrt(velocities[:, 0]**2 + velocities[:, 1]**2)
    min_abs, max_abs = np.min(abs_velocities, axis=0), np.max(abs_velocities, axis=0)
    frames = os.listdir(foldername)
    frame = cv2.imread(rf"{foldername}/{frames[0]}", cv2.IMREAD_GRAYSCALE)
    image = ax.imshow(frame, cmap='gray')

    x0, y0, vx0, vy0 = results[0]
    vy0 = -vy0
    vectors = ax.quiver(x0, y0, vx0, vy0, max_abs-min_abs, cmap='jet', scale=scale, scale_units='xy')
    def update(index):
        frame = cv2.imread(rf"{foldername}/{frames[index]}", cv2.IMREAD_GRAYSCALE)
        image.set_data(frame)

        vx, vy = results[index][2], results[index][3]
        vy = -vy
        abs_vel = np.sqrt(vx**2 + vy**2)
        vectors.set_UVC(vx, vy, abs_vel-min_abs)

        return image, vectors

    _ = animation.FuncAnimation(fig=fig, func=update, frames=results.shape[0], interval=1000/playback_fps)

    plt.show()

In [60]:
def all_frames(window_size, overlap, match_mode = 0, interp_mode = 0):
    results = []
    for i in range(50, 51, 1):
        image_a, image_b = dataset[i]
        
        n_rows, n_cols = get_field_shape(image_a.shape, window_size, overlap)
        window = moving_window_array(image_a, window_size, overlap)
        
        area = moving_reference_array(image_b, window_size, overlap, left=10, right=10, top=10, bottom=10)
        correl = block_match(window, area, match_mode)
        u, v = correlation_to_displacement(correl, n_rows, n_cols, interp_mode)

        x, y= get_x_y(image_a.shape, window_size, overlap)
        x, y = x.reshape(n_rows, n_cols), y.reshape(n_rows, n_cols)
        
        results.append((x, y, u, v))
    return np.array(results)

In [64]:
xx_x = all_frames(64, 32, match_mode=0, interp_mode=0)

100%|██████████| 3969/3969 [00:01<00:00, 2130.01it/s]


torch.Size([3969, 1])
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])


100%|██████████| 3969/3969 [00:01<00:00, 2206.21it/s]


torch.Size([3969, 1])
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])


In [58]:
fn = "../Test Data/plume simulation_PROCESSED"
vector_field_l(fn, xx_x, 0.5)